In [1]:
#Thousands of open source data sets available, but must encode these datas into usable tensors

In [2]:
#Working with a Wine Quality Data Set

In [3]:
#12 columns below from csv range
#First eleven columns have numerical chemical values, last contians sensory quality from 0 - 10 (worst to best)
#fixed acidity
#volatile acidity
#citric acid
#residual sugar
#chlorides
#free sulfur dioxide
#total sulfur dioxide
#density
#pH
#sulphates
#alcohol
#quality

In [4]:
#Possibly use this dataset to help a CNN predict wine quality

In [5]:
import csv
import torch
import numpy as np
wine_path = "../data/winequality-white.csv"
#load csv into numpy array, skip first row as it is all labels
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [6]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [7]:
#convert numpy array into tensor

In [8]:
wineq = torch.from_numpy(wineq_numpy)

In [9]:
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [10]:
#we only need the firt 11 columns not the quality score

In [11]:
data = wineq[:, : -1] #all rows all columns except last 

In [12]:
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [13]:
target = wineq[:, -1].long()

In [14]:
target, target.shape

(tensor([6, 6, 6,  ..., 6, 7, 6]), torch.Size([4898]))

In [15]:
#In this case it may work better to encode the quality scores as a label

In [16]:
#However, in a categorical sense we can use an approach called one-hot encoding

In [17]:
#In this case we could have a vector of 10 elements

In [18]:
#each score would be mapped onto the vector, say a 5 <0,0,0,0,1,0,0,0,0,0>

In [19]:
target_onehot = torch.zeros(target.shape[0], 10)

In [20]:
target_onehot.scatter_(1,target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [21]:
#scatter_, ends with _ so it does not return a new tensor, but will modify the tensor in place

In [22]:
#Arguments for scatter: dimension along which the column tnesor indicating the indices of the elements to scatter

In [23]:
#and the tensor containing the elements to scatter or a single scalar to scatter

In [24]:
target_unsqueezed = target.unsqueeze(1)

In [25]:
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [26]:
#Lets obtain the mean and std of each column
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [27]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [28]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)

In [29]:
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [30]:
#Finding thresholds

In [31]:
#Find which rows in target with a score less or equal to 3 (Bad wines)

In [32]:
bad_indexes = target <= 3 
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [33]:
#PyTorch has advanecd indexing, we can use tensor with bool's to index the data tensor -->Filter data to be only rows with True

In [34]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [35]:
#We can now  group the wines into varying categories (bad, mediocre, good)

In [36]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [38]:
#looking at the data here, we can see that total sulfur dioxide can be used as a criteria for bad vs good wines

In [39]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [40]:
#Above predicts that half of the wines will be considered good

In [41]:
actual_indexes = target > 5

In [42]:
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [44]:
#underprediction of the above average wines
#Lets see under our basic model if the wines that we predicted > 5 will match with the actual

In [45]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

In [ ]:
#we see that there is a 74% chance of predicting a wine to be > 5